In [1]:
from rl.tests.frozen_lake_policy import build_best_policy
from rl.valfuncs.model_free import build_replay_buffer
import gym
import tensorflow as tf
import os.path as path
from rl.valfuncs.hyperparams import Hyperparams
from tensorflow.keras.layers import Input, Dense
import numpy as np
from collections import defaultdict
from scratch_tf import *

In [ ]:
fl = gym.make("FrozenLake-v0")
policy = build_best_policy(fl)
ds = gen_dataset(fl, policy, 1000, 1.0)

In [ ]:
for x, y in ds:
    s = tf.where(tf.equal(x, 1))[0,0].numpy()
    y = y[0].numpy()
    print(s, y)

In [ ]:
print_state_stats(ds)

In [ ]:
run_id = "little-firefly-2047"
model_file = path.expanduser(f"~/mldata/tblogs/frozen-lake/{run_id}/model.h5")
model = tf.keras.models.load_model(model_file)

In [ ]:
for layer in model.layers:
    print(layer.get_weights())

In [2]:
tp = test("little-firefly-2047")



Test Metrics -
1/1 [==============================] - 0s 53ms/step - loss: 0.0084 - rmse: 0.0916
[0] exp_sval=0.017 pred_sval=0.049
[1] exp_sval=0.012 pred_sval=0.030
[2] exp_sval=0.028 pred_sval=0.068
[3] exp_sval=0.012 pred_sval=0.021
[4] exp_sval=0.027 pred_sval=0.068
[6] exp_sval=0.068 pred_sval=0.127
[8] exp_sval=0.064 pred_sval=0.146
[9] exp_sval=0.185 pred_sval=0.346
[10] exp_sval=0.228 pred_sval=0.353
[13] exp_sval=0.325 pred_sval=0.493
[14] exp_sval=0.574 pred_sval=0.660


In [4]:
tp[0]

0.008394668810069561

In [5]:
tp[1]

0.09162243

In [7]:
hparams = Hyperparams(
    batch_size=32,
    epochs=2,
    lr=0.001,
    num_train_steps=100,
    num_val_steps=10,
    activation="tanh",
    regularizer=0,
    discount=1.0,
)

fl = gym.make("FrozenLake-v0")
policy = build_best_policy(fl)
train_ds = gen_dataset(fl, policy, hparams.num_train_steps, hparams.discount)
val_ds = gen_dataset(fl, policy, hparams.num_val_steps, hparams.discount)
model = build_model(fl.observation_space.n, hparams.regularizer)
optim = tf.keras.optimizers.Adam(learning_rate=hparams.lr)
rmse = tf.metrics.RootMeanSquaredError("rmse")
model.compile(optimizer=optim, loss="mse", metrics=[rmse])
train_ds = train_ds.shuffle(hparams.num_train_steps).batch(hparams.batch_size)
val_ds = val_ds.batch(hparams.num_val_steps)
history = model.fit(train_ds, validation_data=val_ds, epochs=hparams.epochs, verbose=2)

Epoch 1/2
3/3 - 1s - loss: 0.3491 - rmse: 0.5812 - val_loss: 0.0000e+00 - val_rmse: 0.0000e+00
Epoch 2/2
3/3 - 0s - loss: 0.3241 - rmse: 0.5774 - val_loss: 0.1417 - val_rmse: 0.3764


In [11]:
history.history

{'loss': [0.3377824658932893, 0.3334127716396166],
 'rmse': [0.5811906, 0.5774191],
 'val_loss': [0.14558681845664978, 0.14167802035808563],
 'val_rmse': [0.38155842, 0.3764014]}

In [12]:
np.min(history.history["val_loss"])

0.14167802035808563

In [13]:
np.argmin(history.history["val_loss"])

1